In [ ]:
from sqlalchemy import *
import pandas as pd
import numpy as np
import random
import getpass

pd.options.display.max_rows = 99999
pd.options.display.max_columns = 99999

In [ ]:
# setup engine to archbox psql database
user='ahendel'
passwd=getpass.getpass('Enter Passwd: ')

In [ ]:
arch_engine = create_engine("postgresql://{}:{}@192.168.0.2:5432/sampalytics".format(user,passwd))

In [ ]:
# get list of tickers in our db
tickers = arch_engine.engine.execute('SELECT DISTINCT ticker from stocks.minute').fetchall()
# convert to list, ie get the tickers out of the tuples
tickers = [x[0] for x in tickers]

In [ ]:
outcome = 'AMD'
n_predictors = 100

In [ ]:
def prune_list(outcome, ticks, n_predictors):
    # remove the outcome var so we dont randomly select it and create duplicates
    ticks = [x for x in ticks if x not in outcome] # .remove causes pointer issue
    # convert list of tickers to df so we can index
    df = pd.DataFrame(ticks)
    # randomly select the rows we want to take
    rows = random.sample(range(len(ticks)), n_predictors)
    # then select the rows subset
    df = df.iloc[rows]
    return [outcome] + list(df[0])

In [ ]:
tickers = prune_list(outcome=outcome, ticks=tickers, n_predictors=n_predictors)

In [ ]:
# custom header
full_df = pd.DataFrame(columns=['dtg'])
# predictors = ['open', 'high', 'low', 'close', 'volume']
predictors = ['close', 'volume'] # try less to save memory
i = 1
for ticker in tickers:
    header = ['dtg'] + [x + '_' + str(ticker) for x in predictors]
    select=', '.join(['dtg'] + predictors)
    # d = arch_engine.engine.execute("SELECT DISTINCT dtg, open, high, low, close, volume from stocks.minute WHERE ticker='{}'".format(ticker)).fetchall()
    d = arch_engine.engine.execute("SELECT DISTINCT {} from stocks.minute WHERE ticker='{}'".format(select, ticker)).fetchall()
    d = pd.DataFrame(d, columns=header).dropna()
    print('Loading: {}: {}'.format(ticker, i))
    i = i + 1
    full_df = pd.merge(full_df, d, on='dtg', how='outer')

In [ ]:
# ie. row 1 is Jan 1, row 2 in Jan 2.
full_df.sort_values('dtg', inplace=True)
# and do a forward fill
full_df.fillna(method='ffill', inplace=True)
# then drop the NA
full_df.dropna(inplace=True)
# rename dtg to date for TS package
full_df.rename(columns={'dtg':'Date'}, inplace=True)

In [ ]:
full_df.to_csv('/mnt/ubudata/projects/data/AMD_plus_100.csv',index=None)

In [ ]:
# continue in main-notebook of forecast engine